<a href="https://colab.research.google.com/github/david-cuervo/PySpark/blob/main/Word_Count_with_MapReduce_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Word Count with MapReduce - Lab

In [1]:
!apt update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,372 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backport

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.8.0-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [5]:
import findspark
findspark.init()

Word Count

We will illustrate a MapReduce computation for counting the number of occurrences for each word in a text corpus. In this example, the input file is a repository of documents and each document is an element. We shall count the frequency of stop words for style identification as stop words might have unique features which can potentially describe the author's writing style based on their use of stop words while writing. We shall look at some texts by Shakespeare and Jane Austin following this motivation.

MapReduce in PySpark provides a practical and efficient way of achieving this goal as it:

* works if the file is too large for memory

* works even if the output is too large for memory

* is naturally parallelizable

MapReduce Framework

Here are the steps that we will perform for our problem, under the MapReduce framework:

* Sequentially read a lot of data (text files in this case)

* Map:

 * Extract something you care about

* Group by key: Sort and shuffle

* Reduce:

 * Aggregate, summarize, filter or transform

* Write the result

Initialize SparkContext()

* First, import the pyspark module into this Python environment and initialize a SparkContext()
* Initialize a local spark context

In [6]:
# Start a local SparkContext
import pyspark
pyspark
sc = pyspark.SparkContext('local[*]') # [*] represents a local context i.e. no cluster

To test our code, start with a single text file, 'hamlet.txt'. First, set a file path variable file to the location of 'text/hamlet.txt'.

In [7]:
# Set a path variable for data 
file = 'text/hamlet.txt'
file

'text/hamlet.txt'

Read and Split text file contents into RDD - sc.textFile()

Previously we used parallelization to read an RDD from a Python list. Here we shall read the text file into Spark RDDs by using sc.textFile() method for loading the text file into the lines RDD. The documentation on RDDs can be found here!!

The textFile(path) method reads a text file from the HDFS/local file system/any Hadoop supported file system, into the number of partitions specified and returns it as an RDD of strings. In order to view the contents of the RDD, we will use the RDD.collect() method since calling the RDD by name will not return the contents, only the object type and relevant information.

In [8]:
# Read the text file into an RDD using sc.textFile()
lines = sc.textFile(file)
lines

text/hamlet.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

The text file has been written in a "line-by-line" manner into the RDD. We can access any given entry using simple indexing.

* Print a few sample lines from the lines RDD

In [9]:
print(lines.collect()[500])
print(lines.collect()[1000])

Py4JJavaError: ignored